### Imports

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_regression
pd.set_option('display.max_columns', 500)

### Load Base Data

In [11]:
energy = pd.read_csv('../data/energy_dataset.csv')
weather = pd.read_csv('../data/weather_features.csv')
weather = weather.drop(columns=['weather_id', 'weather_icon']) # remove these columns; they're not informative

### Avg Data Across Cities

In [12]:
weather_main_dummies = pd.get_dummies(weather.weather_main)
weather = weather.drop(columns=['weather_main', 'weather_description']).join(weather_main_dummies)

In [13]:
total = pd.DataFrame()

for name in weather.city_name.unique():
    cityData = weather.loc[weather.city_name == name]
    dfn = pd.merge(energy, cityData, left_on='time', right_on='dt_iso')
    dfn=dfn.drop_duplicates("time")
    dfn = dfn.set_index("time")
    if(not(total.empty)):
        total+=dfn
    else:
        total=dfn
    print(total.shape)

(35064, 53)
(35064, 53)
(35064, 53)
(35064, 53)
(35064, 53)


In [14]:
# use idxmax to get the most popular value due to the average (can't really avg categorical value in traditional sense)
total['weather_main'] = total[['clear', 'clouds', 'drizzle', 'dust', 'fog', 'haze',
       'mist', 'rain', 'smoke', 'snow', 'squall', 'thunderstorm']].idxmax(axis=1)
total = total.drop(columns=['clear', 'clouds', 'drizzle', 'dust', 'fog', 'haze',
       'mist', 'rain', 'smoke', 'snow', 'squall', 'thunderstorm'])

# take average
def take_avg(col):
    if col.dtype != object:
        col /= 5
    return col

total = total.apply(take_avg)

In [15]:
# don't need these
total = total.drop(columns=['dt_iso', 'city_name'])

In [16]:
# these are null for every datapoint
total = total.drop(columns=['generation hydro pumped storage aggregated', 'forecast wind offshore eday ahead'])

In [17]:
def parse_hour(item):
    hour = int(item[11:13])
    if hour >= 6 and hour < 12:
        return 'morning'
    elif hour >= 12 and hour <=17:
        return 'day'
    return 'night'
def parse_month(item):
    month = int(item[5:7])
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'fall'

In [18]:
# OHE the weather main column
total = pd.get_dummies(total)

In [19]:
# remove 36 entries that have any null values for total load (we don't lose much (?))
total = total.dropna()

In [20]:
# Feature engineer the time of day and season features
total = total.reset_index()
total['time_of_day'] = total.time.apply(parse_hour)
total['season'] = total.time.apply(parse_month)

In [21]:
# push time back into the index for ease of OHE
total = total.set_index('time')

In [22]:
total = pd.get_dummies(total)

In [23]:
total.head(3)

,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation marine,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_main_clear,weather_main_clouds,weather_main_drizzle,weather_main_fog,weather_main_mist,weather_main_rain,weather_main_thunderstorm,time_of_day_day,time_of_day_morning,time_of_day_night,season_fall,season_spring,season_summer,season_winter
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,863.0,1051.0,1899.0,0.0,7096.0,43.0,73.0,49.0,196.0,0.0,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41,272.491463,272.491463,272.491463,1016.4,82.4,2.0,135.2,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,920.0,1009.0,1658.0,0.0,7096.0,43.0,71.0,50.0,195.0,0.0,5890.0,16.0,5856.0,24934.0,24382.0,48.10,64.92,272.512700,272.512700,272.512700,1016.2,82.4,2.0,135.8,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,1164.0,973.0,1371.0,0.0,7099.0,43.0,73.0,50.0,196.0,0.0,5461.0,8.0,5454.0,23515.0,22734.0,47.33,64.48,272.099137,272.099137,272.099137,1016.8,82.0,2.4,119.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,1,0,0,0,1


In [24]:
# NOTE: not all of the items I include are used for prediction of price
total.to_csv("final_baseline_data.csv")